In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import time
import re
from tqdm.notebook import tqdm

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime ,timedelta , date

### __크롬 드라이버 연동__

In [ ]:
my_path = 'D:/downloads/'
driver = webdriver.Chrome(my_path+'chromedriver.exe')
time.sleep(2)

# Fox 뉴스

Load more 버튼을 cnt만큼 눌러서 모든 기사를 페이지에 로딩하는 함수

In [ ]:
def load_all_news(driver,cnt):
    while cnt:
        try:
            driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[2]/div/main/section/footer/div/a').click()
            time.sleep(1.5)
        except Exception as E:
            print(E)
            break
        cnt-=1
    return

각 기사 태깅 후에 `12 hours ago` 와 같은 정보로 부터 날짜를 계산하는 함수:  
- 'age' 형식은 현재 시간에서 빼는 식으로  
- 'October 24' 형식은 그대로 적용

In [ ]:
def get_date(tag): # 뉴스의 날짜 post date 계산
    global current_time
    
    posted_time_str = tag.select('div.info > header > div > div > span.time')[0].text
    
    if 'ago' in posted_time_str :
        
        time_diff = re.search('\d+',posted_time_str).group()
        
        if 'day' in posted_time_str:
            posted_time = current_time - timedelta(days = int(time_diff))
        elif 'hour' in posted_time_str:
            posted_time = current_time - timedelta(hours = int(time_diff))
        elif 'min' in posted_time_str:
            posted_time = current_time - timedelta(minutes = int(time_diff))
        else:
            print('error from 1')

    else:
        
        post_date = re.search('\d+',posted_time_str).group()
        
        if 'tober' in posted_time_str:
            posted_time = datetime(2020,10,int(post_date))
        elif 'tember' in posted_time_str:
            posted_time = datetime(2020,9,int(post_date))
        else:
            print('error from 2')
    
    return date.isoformat(posted_time)

검색 파라미터 설정

In [ ]:
current_time = datetime.now()
data=pd.DataFrame()
load_more = 5

메인 코드

In [ ]:
driver.get('https://www.foxnews.com/category/politics/2020-presidential-election')
time.sleep(2)

load_all_news(driver,load_more)

html = driver.page_source
soup = BeautifulSoup(html,'lxml')
articles = soup.select('#wrapper > div.page > div.page-content > div > main > section > div > article')

news = [ {'headlines':article.select('div.info > header > h4 > a')[0].text,
         'date':get_date(article),
         'Press':'Fox',
         'Link':'https://www.foxnews.com'+article.select('div.info > header > h4 > a')[0].attrs['href'],
         'category':article.select('div.info > header > div > div > span.eyebrow > a')[0].text} for article in articles]
news_df = pd.DataFrame(news)
data = pd.concat([data,news_df])

In [ ]:
data

In [ ]:
data.date.value_counts()

In [ ]:
data.category.value_counts()

수집된 링크들로부터 각각 본문 내용 수집

In [ ]:
body_text_list=[]
for url in data['Link']:

    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    body = soup.select('#wrapper > div.page-content > div.row.full > main > article > div > div.article-content > div.article-body')
    body_text = ''.join([p.text for p in body[0].select('p')])
    body_text_list.append(body_text)
    
data['text'] = pd.Series(body_text_list)

In [ ]:
data.head(2)

# CBS & POLITICO 뉴스

메인 로직 (CBS)

In [ ]:
def cbs_news(page_num):
    
    content=[]
    title=[]
    date=[]
    
    driver = webdriver.Chrome(path+"chromedriver.exe")
    time.sleep(2)
    
    for i in range(1,page_num+1):
        
        driver.get(f'https://www.cbsnews.com/feature/election-2020/{i}/')
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        soup=soup.select('div.col-8.nocontent')[0].select('section')
        
        try:
            for j in range(1,2): #26

                title.append(soup[0].select('h4.item__hed')[j-1].text.strip())
                date.append(soup[0].select('span.item__date')[0].text)

                driver.find_element_by_xpath(f'//*[@id="component-election-2020"]/div/article[{j}]/a').click()
                time.sleep(2)

                html_content = driver.page_source
                soup_content = BeautifulSoup(html_content, "lxml")

                lis=soup_content.select('article')[0].select('p')

                if lis[0].text.strip()[:2]=='By':
                    con=''
                    for k in lis[2:]:
                        con+=k.text.strip() 

                else:
                    con=''
                    for k in lis:
                        con+=k.text.strip()


                content.append(con)
                driver.back()
                time.sleep(2)
        except:
            pass
        
    df=pd.DataFrame({'date':date,'title':title,'content':content})
    return df

메인코드 (CBS)

In [ ]:
data=cbs_news(5)

In [ ]:
n=0
data['date']=data['date'].str.replace('Oct','2020 10').replace('Sep','2020 09')
data['date'][0:n+1]=datetime.date.today()
data['date']=pd.to_datetime(data['date'])
data

메인로직 (POLITICO)

In [ ]:
def politico_news(page_num):
    
    content=[]
    title=[]
    date=[]

    driver = webdriver.Chrome(path+"chromedriver.exe")
    time.sleep(2)

    for i in range(1,page_num+1):

        driver.get(f'https://www.politico.com/news/2020-elections/{i}')
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")

        try:
            for j in range(1,20): #20

                driver.find_element_by_xpath(f'//*[@id="main"]/div[3]/div/div/section[1]/div[2]/ol/li[{j}]/article/div/div/div/div/header/h1/a').click()
                time.sleep(2)

                html_content = driver.page_source
                soup_content = BeautifulSoup(html_content, "lxml")

                lis=soup_content.select('article')[0].select('p')

                con=''
                for k in lis:
                    con+=k.text.strip() 

                content.append(con)
                driver.back()
                time.sleep(2)

                title.append(soup.select('ol.front-list')[0].select('h1')[j-1].text.strip())
                d=soup.select('ol.front-list')[0].select('p.timestamp')[j-1].text[:10]

                if d=='31/08/2020':
                    break

                date.append(d)

        except:
            pass

    df=pd.DataFrame({'date':date,'title':title,'content':content})
    return df

메인코드 (POLOTICO)

In [ ]:
data=politico_news(227)

# NRP & Reuters 뉴스

메인 코드 (NRP)

In [ ]:
driver.get("https://www.npr.org/sections/elections/") # get page

n = 0
while 1 :
    driver.find_element_by_css_selector("#main-section > div.options.has-more-results > button").click() # load 15 times
    n += 1
    if n > 14:
        break

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
link_html = soup.select("div.item-info-wrap > div > h2 > a")  # get links
links = []
for link in link_html:
    links.append(link.get('href'))

    
df = []               # crawl data from every link
for link in links:
    driver.get(link)
    tm.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try : 
        title = soup.select("#main-section > article > div.storytitle > h1")[0].text # 기사제목
        date = soup.select("#story-meta > div.story-meta__one > div > time > span")[0].text # 기사 날짜
        time = soup.select("#story-meta > div.story-meta__one > div > time > span")[1].text #기사 시간
        textall = soup.select("#storytext > p") # 기사 내용

        text = []
        for t in textall :
            text.append(t.text)
            
        topic = soup.select("#main-section > article > div.slug-wrap > h3 > a")[0].text # 주제

    except:
        title = np.nan
        date = np.nan
        time = np.nan
        text = np.nan

    article = {"Date" : date, "Time": time, "Press" : "NPR", "Topic": topic, "Title": title, "Text" : text} # 합치기 
    df.append(article)

In [ ]:
npr = pd.DataFrame(df)
npr.dropna(inplace = True)
npr.drop_duplicates(["Title"])
npr[npr["Date"].str.contains('August')].index
npr = npr.loc[:340, :]
npr.to_csv("nprdata.csv")

메인코드 (Reuters)

In [ ]:
driver.get("https://www.reuters.com/news/archive/us-elections-2020?view=page&page=1&pageSize=10") # get page

links = []
for page in range(1,88):
    url = f"https://www.reuters.com/news/archive/us-elections-2020?view=page&page={page}&pageSize=10" # get links
    driver.get(url)
    tm.sleep(4)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    link_html = soup.select("div > div.column1.col.col-10 > section > section > div > article > div.story-content > a")
    for link in link_html:
        links.append("https://www.reuters.com/" + link.get('href'))
        
        
df = []                     # crawl data from every link
for link in links:
    driver.get(link)
    tm.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try :
        # 기사제목 
        title = soup.select("#__next > div > div.TwoColumnsLayout-hero-3H8pu > div.TwoColumnsLayout-content-2D7Bp.TwoColumnsLayout-left-column-CYquM.ArticlePage-hero-container-1BcoN > div > div > h1")[0].text
        # 기사 날짜
        date = soup.select("#__next > div > div.TwoColumnsLayout-hero-3H8pu > div.TwoColumnsLayout-content-2D7Bp.TwoColumnsLayout-left-column-CYquM.ArticlePage-hero-container-1BcoN > div > div > div > div > time")[0].text
        #기사 시간
        time = soup.select("#__next > div > div.TwoColumnsLayout-hero-3H8pu > div.TwoColumnsLayout-content-2D7Bp.TwoColumnsLayout-left-column-CYquM.ArticlePage-hero-container-1BcoN > div > div > div > div > time")[1].text
        # 주제
        topic = soup.select("#__next > div > div.TwoColumnsLayout-hero-3H8pu > div.TwoColumnsLayout-content-2D7Bp.TwoColumnsLayout-left-column-CYquM.ArticlePage-hero-container-1BcoN > div > div > div > a")[0].text
        # 기사 내용
        textall = soup.select("#__next > div > div.TwoColumnsLayout-body-86gsE.ArticlePage-body-container-10RhS > div.TwoColumnsLayout-left-column-CYquM > article")
        text = []
        for t in textall :
            text.append(t.text)
    except:
        title = np.nan
        date = np.nan
        time = np.nan
        text = np.nan     
        topic = np.nan

    # 합치기 
    article = {"Date" : date, "Time": time, "Press" : "Reuters", "Topic": topic, "Title": title, "Text" : text}
    df.append(article)
    

In [ ]:
rt = pd.DataFrame(df)
rt.dropna(inplace = True)
rt[rt["Date"].str.contains('August')]
rt = rt.loc[:868, :]
rt[rt["Date"].str.contains('November')]
rt = rt.loc[36:, :]

중도 데이터 통합 코드

In [ ]:
center = pd.concat([npr, rt])
center.dropna(inplace = True)
center.to_csv("center.201103.csv",index=False)